## Location

In [ ]:
import pandas as pd

In [ ]:
location = pd.read_csv('SHB_data_luisa/locationeventpertime.csv')

In [ ]:
location.head()

In [ ]:
location = location.drop(["experimentid", "userid", "lucene", "speed", "provider", "bearing"], axis=1)

In [ ]:
len(location)

In [ ]:
type(location.point[0])

## Extract latitude and longitude from points

In [ ]:
lat = []
long = []
for el in location.point:
    lst = el.split('=')
    lat.append(float(lst[1].split(',')[0]))
    long.append(float(lst[2].split(',')[0]))
    
location['lat'] = lat
location['long'] = long

In [ ]:
location

In [ ]:
location = location.drop(["point", "day"], axis=1)

In [ ]:
min(location.accuracy), max(location.accuracy) # depending on the position, different signals thus different accuracy

In [ ]:
location = location.drop(["accuracy"], axis=1)

## Recode longitude and latitude (rounding 3 decimals)

In [ ]:
location.lat = round(location.lat, 3)
location.long = round(location.long, 3)

## Timestamp recoding

In [ ]:
location['timestamp'].isnull().values.any()

In [ ]:
# type(location.timestamp[0])

In [ ]:
# int(round(location.timestamp[0]/100000, 0))

In [ ]:
# location.timestamp = location.timestamp.apply(lambda x: int(round(x/100000, 0)))
# location

In [ ]:
# remove seconds and decimals
location.timestamp = location.timestamp.astype(str)
location.timestamp = location.timestamp.apply(lambda x: x[0:12])
location

## Removing duplicated rows

In [ ]:
location = location.drop_duplicates()

In [ ]:
len(location)

In [ ]:
len(location.timestamp.unique()) # looks like some timestamps are repeated but in different locations
# different location in the same moment???

In [ ]:
location.timestamp.value_counts()

In [ ]:
location[location.timestamp == '202110221831'] # checking latitude and longitude corresponding to repeated timestamps
# basically, they define the same location, just some decimals of difference

In [ ]:
# extract subset of the dataframe with all timestamps appearing more than once
v = location.timestamp.value_counts()
location[location.timestamp.isin(v.index[v.gt(1)])] # get all items (timestamps) appearing more than 1

Substitute repeated timestamps with a unique timestamp, having as latitude and longitude the mean of the latitude and longitude associated to that timestamp.

In [ ]:
v = location.timestamp.value_counts()
for time in location.timestamp:
    latitude = []
    longitude = []
    if time in list(location[location.timestamp.isin(v.index[v.gt(1)])].timestamp):
        latitude.append(location.loc[location['timestamp'] == time, 'lat'].iloc[0]) # take lats where timestamp=to the considered repeated time
        longitude.append(location.loc[location['timestamp'] == time, 'long'].iloc[0])
        location = location[location.timestamp != time] # remove all rows with that timestamp
        import statistics
        mean_lat = round(statistics.mean(latitude), 3)
        mean_long = round(statistics.mean(longitude), 3)
        # add a new row with that timestamp and mean of latitude and longitude as lat and long (rounded at 2 decimals)
        location = location.append({'timestamp': time, 'lat': mean_lat, 'long': mean_long}, ignore_index=True)

In [ ]:
location

In [ ]:
len(location.timestamp.unique())

In [ ]:
location = location.sort_values(by=['timestamp'], ascending=True) 

In [ ]:
# save
location.to_csv('processed_location.csv')

## Get city name from coordinates

In [ ]:
location = pd.read_csv('processed_location.csv')

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
# initialize Nominatim API 
geolocator = Nominatim(user_agent="geoapiExercises")

The following chunk takes approximately one hour to run.

In [ ]:
place = []
for row in range(len(location)):
    place.append(geolocator.reverse(str(location.lat[row])+","+str(location.long[row])))
location['City'] = place

## Save processed dataset

In [ ]:
location.to_csv('processed_location_city.csv')